# AllenNLP tests

Use AllenNLP's semantic role labeling in combination with manually written rules for extracting causal relations from political speeches. We found two limitations: only relations within a single sentence were found and complex relations involving understanding the sentence were hard to identify.

Links:

* software installation: https://github.com/allenai/allennlp (do not forget to install NLTK popular models)
* software usage: https://demo.allennlp.org/semantic-role-labeling (tab: Model Usage)

In [1]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
import re

In [2]:
srl_predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")

/home/erikt/anaconda3/envs/allennlp/lib/python3.7/site-packages/allennlp/tango/__init__.py:18: UserWarning: AllenNLP Tango is an experimental API and parts of it might change or disappear every time we release a new version.
  "AllenNLP Tango is an experimental API and parts of it might change or disappear "
2021-11-16 17:48:58,221 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2021-11-16 17:48:58,497 - INFO - cached_path - cache of https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz is up-to-date
2021-11-16 17:48:58,500 - INFO - allennlp.models.archival - loading archive file https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz from cache at /home/erikt/.allennlp/cache/b5f1db011cc85691a5fa2bf29e055a712261a2e5d74a74edd7da2fffc98d4ab8.4c4ac7e06ec3d85631bd26b839f90b5a375d3ceeb43e3c74f1cf4758dcee2bb3
2021-11-16 17:48:58,505 - INFO - allennlp.models.archival - extractin

In [3]:
COLOR_CONCEPT_1 = 1
COLOR_CONCEPT_2 = 4
COLOR_CONTENT_RELATION_EXPLANATION = 0
NO_COLOR = -1
RELATION_VERBS = [ "allowed", "avoid", "contribute", "eliminated", "entailed", "eroded", "failed", "weakened", ]


def strip_tag(token):
    if len(token) > 1:
        if re.search("^\[", token):
            token = re.sub("^\[", "", token)
            token = re.sub(":$", "", token)
        token = re.sub("\]$", "", token)
    return token


def print_with_color(string, color_code):
    print(f"\x1b[1;3{color_code};47m{string} \x1b[m", end="")
    
    
def print_argm_prp(tokens, arguments, i):
    if i in arguments["ARGM-PRP"]:
        color_code = COLOR_CONCEPT_2
    elif "V" in arguments and i in arguments["V"]:
        color_code = COLOR_CONCEPT_1
    elif "ARG0" in arguments and i in arguments["ARG0"]:
        color_code = COLOR_CONCEPT_1
    elif "ARG1" in arguments and i in arguments["ARG1"]:
        color_code = COLOR_CONCEPT_1
    elif "ARG2" in arguments and i in arguments["ARG2"]:
        color_code = COLOR_CONCEPT_1
    else:
        color_code = NO_COLOR
    if color_code >= 0:
        print_with_color(tokens[i], color_code)
    else:
        print(f"{tokens[i]} ", end="")
        

def print_v(tokens, arguments, i):
    if "V" in arguments and i in arguments["V"]:
        color_code = COLOR_CONTENT_RELATION_EXPLANATION
    elif "ARG0" in arguments and i in arguments["ARG0"]:
        color_code = COLOR_CONCEPT_1
    elif "ARG1" in arguments and i in arguments["ARG1"]:
        color_code = COLOR_CONCEPT_2
    elif "ARG2" in arguments and i in arguments["ARG2"]:
        color_code = COLOR_CONCEPT_2
    else:
        color_code = NO_COLOR
    if color_code >= 0:
        print_with_color(tokens[i], color_code)
    else:
        print(f"{tokens[i]} ", end="")

        
def pretty_print(analyzed_sentence):
    arguments = {}
    tokens = analyzed_sentence.split()
    current_argument = ""
    for i in range(0, len(tokens)):
        if re.search("^\[", tokens[i]):
            current_argument = strip_tag(tokens[i])
        if current_argument != "":
            if current_argument in arguments:
                arguments[current_argument].append(i)
            else:
                arguments[current_argument] = [i]
        if re.search("\]$", tokens[i]):
            current_argument = ""
    for i in range(0, len(tokens)):
        if "ARGM-PRP" in arguments:
            print_argm_prp(tokens, arguments, i)
        elif "V" in arguments and strip_tag(tokens[arguments["V"][-1]]) in RELATION_VERBS:
            print_v(tokens, arguments, i)
        else:
            print(f"{tokens[i]} ", end="")
    print("")

In [4]:
def analyze(sentence):
    analysis = srl_predictor.predict(sentence=sentence)
    for verb_data in analysis['verbs']:
        pretty_print(verb_data['description'])

In [5]:
analyze("You, in Greece, with our support, need to rebuild your country, your structures, your administration, your economy to increase the competitiveness of Greece.")

[ARG0: You] , [ARGM-LOC: in Greece] , [ARGM-MNR: with our support] , [V: need] [ARG1: to rebuild your country , your structures , your administration , your economy to increase the competitiveness of Greece] . 
[ARG0: You] , in Greece , with our support , need to [V: rebuild] [ARG1: your country , your structures , your administration , your economy] [ARGM-PRP: to increase the competitiveness of Greece] . 
You , in Greece , with our support , need to rebuild your country , your structures , your administration , your economy to [V: increase] [ARG1: the competitiveness of Greece] . 


In [6]:
analyze("And the best hope of a return to growth and job creation is inside the euro area.")

[ARGM-DIS: And] [ARG1: the best hope of a return to growth and job creation] [V: is] [ARG2: inside the euro area] . 


In [7]:
analyze("To conclude, let me say a few words on the euro area more generally.")

To [V: conclude] , let me say a few words on the euro area more generally . 
[ARGM-PRP: To conclude] , [V: let] [ARG1: me say a few words on the euro area more generally] . 
To conclude , let [ARG0: me] [V: say] [ARG1: a few words] [ARGM-LOC: on the euro area] [ARGM-ADV: more generally] . 


In [8]:
analyze("We have taken important, fundamental decisions over the last couple of months to safeguard the stability of the euro area, and indeed we are now in the phase of implementation. ")

We [V: have] taken important , fundamental decisions over the last couple of months to safeguard the stability of the euro area , and indeed we are now in the phase of implementation . 
[ARG0: We] have [V: taken] [ARG1: important , fundamental decisions] [ARGM-TMP: over the last couple of months] [ARGM-PRP: to safeguard the stability of the euro area] , and indeed we are now in the phase of implementation . 
[ARG0: We] have taken important , fundamental decisions over the last couple of months to [V: safeguard] [ARG1: the stability of the euro area] , and indeed we are now in the phase of implementation . 
We have taken important , fundamental decisions over the last couple of months to safeguard the stability of the euro area , and [ARGM-ADV: indeed] [ARG1: we] [V: are] [ARGM-TMP: now] [ARG2: in the phase of implementation] . 


In [9]:
analyze("""A number of governments have embarked on a path of reform and fiscal consolidation that was unthinkable only very recently, 
        and they have taken important decisions and I encourage them to keep this determination.""")

A number of governments [V: have] embarked on a path of reform and fiscal consolidation that was unthinkable only very recently , and they have taken important decisions and I encourage them to keep this determination . 
[ARG0: A number of governments] have [V: embarked] [ARG1: on a path of reform and fiscal consolidation that was unthinkable only very recently] , and they have taken important decisions and I encourage them to keep this determination . 
A number of governments have embarked on [ARG1: a path of reform and fiscal consolidation] [R-ARG1: that] [V: was] [ARG2: unthinkable] [ARGM-TMP: only very recently] , and they have taken important decisions and I encourage them to keep this determination . 
A number of governments have embarked on a path of reform and fiscal consolidation that was unthinkable only very recently , and they [V: have] taken important decisions and I encourage them to keep this determination . 
A number of governments have embarked on a path of reform and 

In [10]:
analyze("These reforms are now being implemented and this effort must continue with credibility, with consistency, with coherence over time. ")

These reforms [V: are] now being implemented and this effort must continue with credibility , with consistency , with coherence over time . 
These reforms are now [V: being] implemented and this effort must continue with credibility , with consistency , with coherence over time . 
[ARG1: These reforms] are [ARGM-TMP: now] being [V: implemented] and this effort must continue with credibility , with consistency , with coherence over time . 
These reforms are now being implemented and this effort [V: must] continue with credibility , with consistency , with coherence over time . 
These reforms are now being implemented and [ARG1: this effort] [ARGM-MOD: must] [V: continue] [ARGM-MNR: with credibility , with consistency , with coherence over time] . 


In [11]:
analyze("As we said there will not be magic solutions.")

As [ARG0: we] [V: said] there will not be magic solutions . 
As we said there [V: will] not be magic solutions . 
[ARGM-ADV: As we said] there [ARGM-MOD: will] [ARGM-NEG: not] [V: be] [ARG1: magic solutions] . 


In [12]:
analyze("We need sustained efforts and determination.")

[ARG0: We] [V: need] [ARG1: sustained efforts and determination] . 


In [13]:
analyze("At the same time, the existing financial backstops are being used as necessary.")

At the same time , the [V: existing] [ARG1: financial backstops] are being used as necessary . 
At the same time , the existing financial backstops [V: are] being used as necessary . 
At the same time , the existing financial backstops are [V: being] used as necessary . 
[ARGM-TMP: At the same time] , [ARG1: the existing financial backstops] are being [V: used] [ARGM-MNR: as necessary] . 


In [14]:
analyze("Most recently, the financial assistance to the recapitalisation of Spanish banks has been agreed and is ready for implementation.")

Most recently , the financial assistance to the recapitalisation of Spanish banks [V: has] been agreed and is ready for implementation . 
Most recently , the financial assistance to the recapitalisation of Spanish banks has [V: been] agreed and is ready for implementation . 
[ARGM-TMP: Most recently] , [ARG1: the financial assistance to the recapitalisation of Spanish banks] has been [V: agreed] and is ready for implementation . 
[ARGM-TMP: Most recently] , [ARG1: the financial assistance to the recapitalisation of Spanish banks] has been agreed and [V: is] [ARG2: ready for implementation] . 


In [15]:
analyze("""Giving to the ECB the ultimate responsibility for supervision of banks in the euro area will decisively contribute to increase confidence between the banks 
           and in this way increase the financial stability in the euro area.""")

[V: Giving] [ARG2: to the ECB] [ARG1: the ultimate responsibility for supervision of banks in the euro area] will decisively contribute to increase confidence between the banks and in this way increase the financial stability in the euro area . 
Giving to the ECB the ultimate responsibility for supervision of banks in the euro area [V: will] decisively contribute to increase confidence between the banks and in this way increase the financial stability in the euro area . 
[ARGM-PRD: Giving to the ECB the ultimate responsibility for supervision of banks in the euro area] [ARGM-MOD: will] [ARGM-MNR: decisively] [V: contribute] [ARG2: to increase confidence between the banks and in this way increase the financial stability in the euro area] . 
Giving to the ECB the ultimate responsibility for supervision of banks in the euro area will decisively contribute to [V: increase] [ARG1: confidence between the banks] and in this way increase the financial stability in the euro area . 
Giving to th

In [16]:
analyze("Second, Germany is acutely aware of the need to tackle the root causes and not just the symptoms of the crisis.")

[ARGM-DIS: Second] , [ARG1: Germany] [V: is] [ARG2: acutely aware of the need to tackle the root causes and not just the symptoms of the crisis] . 
Second , Germany is acutely aware of the need to [V: tackle] [ARG1: the root causes and not just the symptoms of the crisis] . 


In [17]:
analyze("This is why it is pressing strongly for institutional reforms of the EMU framework plus structural reforms and budgetary discipline in the member states.")

[ARG1: This] [V: is] [ARG2: why it is pressing strongly for institutional reforms of the EMU framework plus structural reforms and budgetary discipline in the member states] . 
This is why it [V: is] pressing strongly for institutional reforms of the EMU framework plus structural reforms and budgetary discipline in the member states . 
This is [ARGM-CAU: why] [ARG0: it] is [V: pressing] [ARGM-MNR: strongly] [ARG2: for institutional reforms of the EMU framework plus structural reforms and budgetary discipline in the member states] . 


In [18]:
analyze("The introduction of the euro eliminated exchange rate risks.")

[ARG0: The introduction of the euro] [V: eliminated] [ARG1: exchange rate risks] . 


In [19]:
analyze("Another implication of the euro area's single monetary policy is that the key interest rates are set for the currency bloc as a whole.")

[ARG1: Another implication of the euro area 's single monetary policy] [V: is] [ARG2: that the key interest rates are set for the currency bloc as a whole] . 
Another implication of the euro area 's single monetary policy is that the key interest rates [V: are] set for the currency bloc as a whole . 
Another implication of the euro area 's single monetary policy is that [ARG1: the key interest rates] are [V: set] [ARGM-ADV: for the currency bloc as a whole] . 


In [20]:
analyze("""One risk is the temptation for governments to overborrow because the economic costs of excessive public debt, 
           for example higher interest rates, can be more easily shifted to other member states.""")

[ARG1: One risk] [V: is] [ARG2: the temptation for governments to overborrow because the economic costs of excessive public debt , for example higher interest rates , can be more easily shifted to other member states] . 
One risk is the temptation for governments to overborrow because the economic costs of excessive public debt , for example higher interest rates , [V: can] be more easily shifted to other member states . 
One risk is the temptation for governments to overborrow because the economic costs of excessive public debt , for example higher interest rates , can [V: be] more easily shifted to other member states . 
One risk is the temptation for governments to overborrow because [ARG1: the economic costs of excessive public debt , for example higher interest rates] , [ARGM-MOD: can] be [ARGM-MNR: more easily] [V: shifted] [ARG2: to other member states] . 


In [21]:
analyze("Even Germany ran up excessive deficits for a few years and, even worse, championed a reform of the SGP which ultimately further weakened the application of the fiscal rules.")

[ARG0: Even Germany] [V: ran] up [ARG1: excessive deficits] [ARGM-TMP: for a few years] and , even worse , championed a reform of the SGP which ultimately further weakened the application of the fiscal rules . 
[ARG0: Even Germany] ran up excessive deficits for a few years and , [ARGM-ADV: even worse] , [V: championed] [ARG1: a reform of the SGP which ultimately further weakened the application of the fiscal rules] . 
Even Germany ran up excessive deficits for a few years and , even worse , championed [ARG0: a reform of the SGP] [R-ARG0: which] [ARGM-TMP: ultimately] [ARG2: further] [V: weakened] [ARG1: the application of the fiscal rules] . 


In [22]:
analyze("However, the EMU framework not only failed to avoid excessive deficits, it was also unable to prevent the build-up of macroeconomic imbalances within the euro area.")

[ARGM-DIS: However] , [ARG1: the EMU framework] not only [V: failed] [ARG2: to avoid excessive deficits] , it was also unable to prevent the build - up of macroeconomic imbalances within the euro area . 
However , [ARG0: the EMU framework] not only failed to [V: avoid] [ARG1: excessive deficits] , it was also unable to prevent the build - up of macroeconomic imbalances within the euro area . 
However , the EMU framework not only failed to avoid excessive deficits , [ARG1: it] [V: was] [ARGM-ADV: also] [ARG2: unable to prevent the build - up of macroeconomic imbalances within the euro area] . 
However , the EMU framework not only failed to avoid excessive deficits , [ARG0: it] was also unable to [V: prevent] [ARG1: the build - up of macroeconomic imbalances within the euro area] . 


In [23]:
analyze("The resulting increase in domestic inflation and wages eroded the competitiveness of the countries concerned and increased their dependence on capital imports.")

The [V: resulting] [ARG2: increase] in domestic inflation and wages eroded the competitiveness of the countries concerned and increased their dependence on capital imports . 
[ARG0: The resulting increase in domestic inflation and wages] [V: eroded] [ARG1: the competitiveness of the countries concerned] and increased their dependence on capital imports . 
The resulting increase in domestic inflation and wages eroded the competitiveness of [ARG1: the countries] [V: concerned] and increased their dependence on capital imports . 
[ARG0: The resulting increase in domestic inflation and wages] eroded the competitiveness of the countries concerned and [V: increased] [ARG1: their dependence on capital imports] . 


In [24]:
analyze("The task of implementing the reforms and regaining competitiveness entailed significant political and social costs.")

The task of [V: implementing] [ARG1: the reforms] and regaining competitiveness entailed significant political and social costs . 
The task of implementing the reforms and [V: regaining] [ARG1: competitiveness] entailed significant political and social costs . 
[ARG0: The task of implementing the reforms and regaining competitiveness] [V: entailed] [ARG1: significant political and social costs] . 


In [25]:
analyze("However, these efforts, supported by a strong expansion in the global economy, allowed German growth to rebound after 2005.")

However , [ARG1: these efforts] , [V: supported] [ARG0: by a strong expansion in the global economy] , allowed German growth to rebound after 2005 . 
[ARGM-DIS: However] , [ARG0: these efforts , supported by a strong expansion in the global economy] , [V: allowed] [ARG1: German growth to rebound after 2005] . 
However , these efforts , supported by a strong expansion in the global economy , allowed [ARG1: German growth] to [V: rebound] [ARGM-TMP: after 2005] . 


In [26]:
analyze("In order to achieve a turnaround and allow further assistance, it is now essential for Greece to deliver on the promises that have been made.")

In order to [V: achieve] [ARG1: a turnaround] and allow further assistance , it is now essential for Greece to deliver on the promises that have been made . 
In order to achieve a turnaround and [V: allow] [ARG1: further assistance] , it is now essential for Greece to deliver on the promises that have been made . 
[ARGM-PRP: In order to achieve a turnaround and allow further assistance] , it [V: is] [ARGM-TMP: now] [ARG2: essential] [ARG1: for Greece to deliver on the promises that have been made] . 
In order to achieve a turnaround and allow further assistance , it is now essential for [ARG0: Greece] to [V: deliver] [ARG1: on the promises that have been made] . 
In order to achieve a turnaround and allow further assistance , it is now essential for Greece to deliver on the promises that [V: have] been made . 
In order to achieve a turnaround and allow further assistance , it is now essential for Greece to deliver on the promises that have [V: been] made . 
In order to achieve a turnar

Modifiers in Propbank (source: http://clear.colorado.edu/compsem/documents/propbank_guidelines.pdf )
* ADJ: Adjectival
* ADV: Adverbials
* CAU: Cause
* COM: Comitative
* DIR: Directional
* DIS: Discourse
* DSP: Direct Speech
* EXT: Extent
* GOL: Goal
* LOC: Locative
* LVB: Light Verb
* MNR: Manner
* MOD: Modal
* NEG: Negation
* PRD: Secondary Predication
* PRP: Purpose
* REC: Reciprocals
* SLC: Relative Clause